In [32]:
import pandas as pd
import numpy as np
from scipy import stats

from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, RandomizedSearchCV

# загружаем класс pipeline
from sklearn.pipeline import Pipeline

# загружаем классы для подготовки данных
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer

# загружаем класс для работы с пропусками
from sklearn.impute import SimpleImputer

# загружаем нужные модели
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# загружаем функцию для работы с метриками
from sklearn.metrics import roc_auc_score, accuracy_score

## Загрузка данных и предобработка признаков

In [5]:
# Основные таблицы
purchases = pd.read_csv('filtered_data/apparel-purchases.csv')  # Данные о покупках
messages = pd.read_csv('filtered_data/apparel-messages.csv')    # Действия по рассылкам
target = pd.read_csv('filtered_data/apparel-target_binary.csv')                # Целевая переменная

# Агрегации по рассылкам
full_event = pd.read_csv('filtered_data/full_campaign_daily_event.csv')           # Агрегация по событиям
full_event_channel = pd.read_csv('filtered_data/full_campaign_daily_event_channel.csv')  # Агрегация по событиям и каналам

In [7]:
purchases.info()
purchases.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202208 entries, 0 to 202207
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   client_id     202208 non-null  int64  
 1   quantity      202208 non-null  int64  
 2   price         202208 non-null  float64
 3   category_ids  202208 non-null  object 
 4   date          202208 non-null  object 
 5   message_id    202208 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 9.3+ MB


,client_id,quantity,price,category_ids,date,message_id
0,1515915625468169594,1,1999.0,"['4', '28', '57', '431']",2022-05-16,1515915625468169594-4301-627b661e9736d
1,1515915625468169594,1,2499.0,"['4', '28', '57', '431']",2022-05-16,1515915625468169594-4301-627b661e9736d
2,1515915625471138230,1,6499.0,"['4', '28', '57', '431']",2022-05-16,1515915625471138230-4437-6282242f27843
3,1515915625471138230,1,4999.0,"['4', '28', '244', '432']",2022-05-16,1515915625471138230-4437-6282242f27843
4,1515915625471138230,1,4999.0,"['4', '28', '49', '413']",2022-05-16,1515915625471138230-4437-6282242f27843


In [8]:
messages.info()
messages.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12739798 entries, 0 to 12739797
Data columns (total 7 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   bulk_campaign_id  int64 
 1   client_id         int64 
 2   message_id        object
 3   event             object
 4   channel           object
 5   date              object
 6   created_at        object
dtypes: int64(2), object(5)
memory usage: 680.4+ MB


,bulk_campaign_id,client_id,message_id,event,channel,date,created_at
0,4439,1515915625626736623,1515915625626736623-4439-6283415ac07ea,open,email,2022-05-19,2022-05-19 00:14:20
1,4439,1515915625490086521,1515915625490086521-4439-62834150016dd,open,email,2022-05-19,2022-05-19 00:39:34
2,4439,1515915625553578558,1515915625553578558-4439-6283415b36b4f,open,email,2022-05-19,2022-05-19 00:51:49
3,4439,1515915625553578558,1515915625553578558-4439-6283415b36b4f,click,email,2022-05-19,2022-05-19 00:52:20
4,4439,1515915625471518311,1515915625471518311-4439-628341570c133,open,email,2022-05-19,2022-05-19 00:56:52


In [9]:
target.info()
target.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49849 entries, 0 to 49848
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   client_id  49849 non-null  int64
 1   target     49849 non-null  int64
dtypes: int64(2)
memory usage: 779.0 KB


,client_id,target
0,1515915625468060902,0
1,1515915625468061003,1
2,1515915625468061099,0
3,1515915625468061100,0
4,1515915625468061170,0


In [10]:
full_event.info()
full_event.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131072 entries, 0 to 131071
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   date                 131072 non-null  object
 1   bulk_campaign_id     131072 non-null  int64 
 2   count_click          131072 non-null  int64 
 3   count_complain       131072 non-null  int64 
 4   count_hard_bounce    131072 non-null  int64 
 5   count_open           131072 non-null  int64 
 6   count_purchase       131072 non-null  int64 
 7   count_send           131072 non-null  int64 
 8   count_soft_bounce    131072 non-null  int64 
 9   count_subscribe      131072 non-null  int64 
 10  count_unsubscribe    131072 non-null  int64 
 11  nunique_click        131072 non-null  int64 
 12  nunique_complain     131072 non-null  int64 
 13  nunique_hard_bounce  131072 non-null  int64 
 14  nunique_open         131072 non-null  int64 
 15  nunique_purchase     131072 non-nu

,date,bulk_campaign_id,count_click,count_complain,count_hard_bounce,count_open,count_purchase,count_send,count_soft_bounce,count_subscribe,...,nunique_open,nunique_purchase,nunique_send,nunique_soft_bounce,nunique_subscribe,nunique_unsubscribe,count_hbq_spam,nunique_hbq_spam,count_close,nunique_close
0,2022-05-19,563,0,0,0,4,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0
1,2022-05-19,577,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2022-05-19,622,0,0,0,2,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
3,2022-05-19,634,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,2022-05-19,676,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
full_event_channel.info()
full_event_channel.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131072 entries, 0 to 131071
Data columns (total 36 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   date                             131072 non-null  object
 1   bulk_campaign_id                 131072 non-null  int64 
 2   count_click_email                131072 non-null  int64 
 3   count_click_mobile_push          131072 non-null  int64 
 4   count_open_email                 131072 non-null  int64 
 5   count_open_mobile_push           131072 non-null  int64 
 6   count_purchase_email             131072 non-null  int64 
 7   count_purchase_mobile_push       131072 non-null  int64 
 8   count_soft_bounce_email          131072 non-null  int64 
 9   count_subscribe_email            131072 non-null  int64 
 10  count_unsubscribe_email          131072 non-null  int64 
 11  nunique_click_email              131072 non-null  int64 
 12  nunique_click_mo

,date,bulk_campaign_id,count_click_email,count_click_mobile_push,count_open_email,count_open_mobile_push,count_purchase_email,count_purchase_mobile_push,count_soft_bounce_email,count_subscribe_email,...,count_send_email,nunique_hard_bounce_email,nunique_hbq_spam_email,nunique_send_email,count_soft_bounce_mobile_push,nunique_soft_bounce_mobile_push,count_complain_email,nunique_complain_email,count_close_mobile_push,nunique_close_mobile_push
0,2022-05-19,563,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-05-19,577,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2022-05-19,622,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2022-05-19,634,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2022-05-19,676,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Поменяем тип данных для дат:

In [13]:
purchases['date'] = pd.to_datetime(purchases['date'])
messages['date'] = pd.to_datetime(messages['date'])
messages['created_at'] = pd.to_datetime(messages['created_at'])
full_event['date'] = pd.to_datetime(full_event['date'])
full_event_channel['date'] = pd.to_datetime(full_event_channel['date'])

## Создание признаков

Создадим набор признаков на уровне клиента:

In [17]:
# Базовая агрегация по клиенту
purchase_features = purchases.groupby('client_id').agg({
    'quantity': ['sum', 'mean'],
    'price': ['sum', 'mean'],
    'date': 'nunique'
})
purchase_features.columns = ['_'.join(col) for col in purchase_features.columns]
purchase_features = purchase_features.reset_index()

# Количество уникальных категорий
purchases['num_categories'] = purchases['category_ids'].apply(lambda x: len(str(x).strip("[]").split(',')))
cat_stats = purchases.groupby('client_id')['num_categories'].agg(['mean', 'max']).reset_index()
purchase_features = purchase_features.merge(cat_stats, on='client_id', how='left')

purchase_features.head()


,client_id,quantity_sum,quantity_mean,price_sum,price_mean,date_nunique,mean,max
0,1515915625468060902,7,1.0,6993.0,999.000000,1,4.000000,4
1,1515915625468061003,7,1.0,16235.0,2319.285714,1,4.000000,4
2,1515915625468061099,1,1.0,299.0,299.000000,1,4.000000,4
3,1515915625468061100,2,1.0,2098.0,1049.000000,1,4.000000,4
4,1515915625468061170,19,1.0,33601.0,1768.473684,3,3.684211,4


In [19]:
# Пивот событий
message_events = messages.pivot_table(index='client_id', 
                                      columns='event', 
                                      values='message_id', 
                                      aggfunc='count', 
                                      fill_value=0).reset_index()

# Кол-во уникальных сообщений
message_counts = messages.groupby('client_id')['message_id'].nunique().reset_index()
message_counts.columns = ['client_id', 'unique_messages']

# Объединение фич
message_features = message_events.merge(message_counts, on='client_id', how='left')
message_features.head()

,client_id,click,close,complain,hard_bounce,hbq_spam,open,purchase,send,soft_bounce,subscribe,unsubscribe,unique_messages
0,1515915625468060902,10,0,0,0,0,35,5,126,0,0,1,127
1,1515915625468061003,6,0,0,0,0,5,1,154,0,0,0,154
2,1515915625468061099,8,0,0,2,0,51,0,215,0,0,0,215
3,1515915625468061100,1,0,0,1,0,163,1,267,1,0,0,268
4,1515915625468061170,16,0,0,0,0,31,3,243,0,0,0,243


In [22]:
# Слияние всех фич и таргета
df = target.merge(purchase_features, on='client_id', how='left')
df = df.merge(message_features, on='client_id', how='left')
df = df.fillna(0)  # Замена пропусков на 0

## Обучение и тестирование модели

In [33]:
# Разделение фич и целевой переменной
X = df.drop(['client_id', 'target'], axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Категориальные и числовые признаки
ohe_columns = X_train.select_dtypes(exclude='number').columns.tolist()
num_columns = X_train.select_dtypes(include='number').columns.tolist()

# Пайплайн для категориальных фичей
ohe_pipe = Pipeline([
    ('simpleImputer_ohe', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first'))
])

# Общий препроцессор
data_preprocessor = ColumnTransformer([
    ('ohe', ohe_pipe, ohe_columns),
    ('num', MinMaxScaler(), num_columns)
], remainder='passthrough')

# Финальный пайплайн (модель будет заменяться)
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=42))  # заглушка, будет заменяться
])

# Грид с XGBoost добавлен
param_grid = [
    {
        'models': [DecisionTreeClassifier(random_state=42)],
        'models__max_depth': range(2, 5),
        'models__max_features': range(2, 5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    },
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 8),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']   
    },
    {
        'models': [SVC(kernel='poly')],
        'models__degree': range(2, 8),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    },
    {
        'models': [LogisticRegression(random_state=42, solver='liblinear', penalty='l1')],
        'models__C': range(1, 5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    },
    {
        'models': [XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')],
        'models__n_estimators': [50, 100, 150],
        'models__max_depth': [3, 5, 7],
        'models__learning_rate': [0.05, 0.1, 0.2],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    }
]

# RandomizedSearchCV с оценкой по ROC AUC
randomized_search = RandomizedSearchCV(
    pipe_final, 
    param_distributions=param_grid, 
    cv=5,
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# Обучение
randomized_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [22:39:18] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('ohe',
                                                                               Pipeline(steps=[('simpleImputer_ohe',
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('ohe',
                                                                                                OneHotEncoder(drop='first',
                                                                                                              handle_unknown='ignore',
                                                                                                              sparse_output=False))]),
                                                                               []),
                                                                              ('num',
                                                                               MinMaxScaler(),
                                                                               ['quantity_sum',
                                                                                'quantity_mean',
                                                                                'pr...
                                                                  missing=nan,
                                                                  monotone_constraints=None,
                                                                  multi_strategy=None,
                                                                  n_estimators=None,
                                                                  n_jobs=None,
                                                                  num_parallel_tree=None,
                                                                  random_state=42, ...)],
                                         'models__learning_rate': [0.05, 0.1,
                                                                   0.2],
                                         'models__max_depth': [3, 5, 7],
                                         'models__n_estimators': [50, 100, 150],
                                         'preprocessor__num': [StandardScaler(),
                                                               MinMaxScaler(),
                                                               'passthrough']}],
                   random_state=42, scoring='roc_auc', verbose=1)

In [35]:
print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели на кроссвалидационной выборке:', randomized_search.best_score_)

Лучшая модель и её параметры:

 Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('simpleImputer_ohe',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  []),
                                                 ('num', MinMaxScaler(),
                                                  ['quantity_sum',
                                             

In [36]:
best_model = randomized_search.best_estimator_
pred = best_model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, pred)
print('Метрика ROC-AUC на тестовой выборке:\n\n', roc_auc)

Метрика ROC-AUC на тестовой выборке:

 0.7371661297470512
